In [9]:
#!pip install pyspellchecker
#!pip install demoji
!pip install autocorrect

     |████████████████████████████████| 622 kB 1.1 MB/s eta 0:00:01
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622380 sha256=759951be7f354ed0921b53ceb161aba9baeba6f49a2be129d1eaa6f3de3b9519
  Stored in directory: /Users/Anna/Library/Caches/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


In [10]:
import pandas as pd

data = pd.read_csv('1_dataset_concatenated.csv')

In [11]:
data.head()

,Реплика 1,Реплика 2
0,"Рио Х в 2022 станет статусным кроссовером, не...",Так и сейчас они могут взять тот же самый хтр...
1,Для тех кто спрашивает что взять за лям или д...,"В 2020м покупал гольфа за 28к евро, когда у ва..."
2,"В 2020м покупал гольфа за 28к евро, когда у ва...","Няма таго, што раньш было…(Я 2015 год вспомин..."
3,"В 2020м покупал гольфа за 28к евро, когда у ва...","А кстати что за гольф, какой двигатель и компл..."
4,"Можешь вообще на изи, просто не хочешь уже бо...",на машину нужно тратить не более 6 зарплат Кто...


In [12]:
len(data)

331279

In [13]:
import re
import demoji
from nltk.corpus import stopwords
from autocorrect import Speller

spell=Speller(lang="ru")
WORD = re.compile(r'\w+')
def reTokenize(doc):
    tokens = WORD.findall(doc)
    return tokens

def spellchecker(text: str) -> str:
    sptext = ' '.join([spell(w).lower() for w in reTokenize(text)])  
    return sptext  

def latin_cyrillic_tokens(text: str) -> str:
    """
    Remove non-latin and non-cyrillic tokens (типо 'anschließend')
    :param text: The text needed to be cleaned.
    :return: Text without non-latin and non cyrillic tokens.
    """
    tokens = text.split(" ")
    result_tokens = []
    for token in tokens:
        try:
            token.encode(encoding="utf-8").decode(encoding="ascii")
        except UnicodeDecodeError:
            if bool(re.search("[а-яА-ЯёЁ]", token)):
                result_tokens.append(token)
        else:
            result_tokens.append(token)

    result_text = " ".join(result_tokens)
    return result_text

def remove_stopwords(text: str) -> str:
    """
    Remove stopwords.
    :param text: The text needed to be cleaned.
    :return: Text without stopwords.
    """
    russian_stopwords = stopwords.words("russian")
    russian_stopwords = [
        word for word in russian_stopwords if word not in ("не", "нет", "да")
    ]
    tokens = [word for word in text.split(" ") if word not in russian_stopwords]
    text = " ".join(tokens)
    return text

def clean_emoji(text: str) -> str:
    """
    Remove emoji tags in the text.
    :param text: The text needed to be cleaned.
    :return: Cleaned text.
    """
    emj_dict = demoji.findall(text)
    for emj in emj_dict.keys():
        try:
            text = re.sub(emj, "", text)
        except KeyError:
            text = text.replace(emj, "")
    emj_img = re.findall(f'(<img class="emoji".*?alt="(.*?)">)', text)
    for img in emj_img:
        text = text.replace(img[0], img[1])
    return text

def preprocessing(text: str) -> str:

    # Make lowercase
    text = text.lower()
    
    # Remove non-latin and non-cyrillic tokens.
    text = latin_cyrillic_tokens(text)
    
    # Remove stopwords.
    # text = remove_stopwords(text)
    
    # Remove numbers 
    text = re.sub(r"\d{1,}", " ", text)
    
    # Clean html tags
    text = re.sub("(<.*?>)", " ", text)
    text = re.sub('(«|»|")', " ", text)

    # Remove URL tags 
    text = re.sub(
        r"((https?):((//)|(\\\\))+([<?>?\w\d:#@%/;$()~_?\+-=\\\.&…](#!)?)*)",
        "",
        text,
    )
    text = re.sub(r"\b([\w\d/]+\.)+[\w\d/]+\b", "", text)

    # Clean text from emoji
    text = clean_emoji(text)
    
    # Remove phone numbers
    text = re.sub(
        r"[7,8]([-\.\s]*\d{3}[-\.\s]*\d{3}[-\.\s]*\d{2}[-\.\s]*\d{2})", " ", text
    )
    
    # Delete repeating spaces
    text = re.sub("\s+", " ", text)
#     for special_char in ".,!?$:\/)(#'\"-_;:”":
#         text = re.sub(f"\{special_char}+", special_char + " ", text)

    # Замена ё на е
    text = text.replace('ё','е')
    
    # Spell checking
    #text = spellchecker(text)
    
    return text

In [14]:
# remove NaN
data = data.dropna()
print(len(data))
data.head(5)

331279


,Реплика 1,Реплика 2
0,"Рио Х в 2022 станет статусным кроссовером, не...",Так и сейчас они могут взять тот же самый хтр...
1,Для тех кто спрашивает что взять за лям или д...,"В 2020м покупал гольфа за 28к евро, когда у ва..."
2,"В 2020м покупал гольфа за 28к евро, когда у ва...","Няма таго, што раньш было…(Я 2015 год вспомин..."
3,"В 2020м покупал гольфа за 28к евро, когда у ва...","А кстати что за гольф, какой двигатель и компл..."
4,"Можешь вообще на изи, просто не хочешь уже бо...",на машину нужно тратить не более 6 зарплат Кто...


In [15]:
data['Реплика 1'] = data['Реплика 1'].apply(
                lambda text: preprocessing(text)
            )

data['Реплика 2'] = data['Реплика 2'].apply(
                lambda text: preprocessing(text)
            )

print(len(data))
data.head(5)

331279


,Реплика 1,Реплика 2
0,"рио х в станет статусным кроссовером, недосту...",так и сейчас они могут взять тот же самый хтр...
1,для тех кто спрашивает что взять за лям или д...,"в м покупал гольфа за к евро, когда у вас евро..."
2,"в м покупал гольфа за к евро, когда у вас евро...","няма таго, што раньш было…(я год вспоминаю и ..."
3,"в м покупал гольфа за к евро, когда у вас евро...","а кстати что за гольф, какой двигатель и компл..."
4,"можешь вообще на изи, просто не хочешь уже бо...",на машину нужно тратить не более зарплат кто т...


In [16]:
data=data[(data['Реплика 1'].apply(lambda x: 25<len(str(x))<500) & (data['Реплика 1'].apply(lambda x: 25<len(str(x))<500)))]
                                                                    

In [17]:
len(data)

273698

In [18]:
data.to_csv('2_preprocessed_data.csv')

In [40]:
# texts_1 = list(data['Реплика 1'])
# texts_2 = list(data['Реплика 2'])

# for x in texts_1+texts_2:
#     print(x,'\n\n', len(x))

In [24]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# corpus = texts_1 + texts_2

# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)
# vectorizer.get_feature_names()

In [7]:
# names = vectorizer.get_feature_names()

In [38]:
# duplicate_indxs = []
# for i, (txt_1, txt_2) in enumerate(zip(texts_1, texts_2)):
#     if type(txt_2) != str:
#         print(type(txt_1))